In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 30 20:11:19 2016

@author: stephen
"""
 
from __future__ import print_function
 
from keras.models import Model
from keras.utils import np_utils
import numpy as np
import pandas as pd
import keras 
from keras.callbacks import ReduceLROnPlateau
      
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 2000
foder = u"E:/Jupyter File/UCR_TS_Archive_2015/"

#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

flist  = ['Adiac']
for each in flist:
    fname = each
    x_train, y_train = readucr(foder+fname+'/'+fname+'_TRAIN')
    x_test, y_test = readucr(foder+fname+'/'+fname+'_TEST')
    nb_classes = len(np.unique(y_test))
    batch_size = min(x_train.shape[0]/10, 16)
    
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
    
    
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
    
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
     
    x_test = (x_test - x_train_mean)/(x_train_std)
    x_train = x_train.reshape(x_train.shape + (1,1,))
    x_test = x_test.reshape(x_test.shape + (1,1,))

    x = keras.layers.Input(x_train.shape[1:])
#    drop_out = Dropout(0.2)(x)
    conv1 = keras.layers.Conv2D(128, 10, 1, border_mode='same')(x)
    conv1 = keras.layers.normalization.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation('relu')(conv1)
    
#    drop_out = Dropout(0.2)(conv1)
    conv2 = keras.layers.Conv2D(256, 8, 1, border_mode='same')(conv1)
    conv2 = keras.layers.normalization.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)
    
#    drop_out = Dropout(0.2)(conv2)
    conv3 = keras.layers.Conv2D(256, 5, 1, border_mode='same')(conv2)
    conv3 = keras.layers.normalization.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)
    
    #    drop_out = Dropout(0.2)(conv2)
    conv4 = keras.layers.Conv2D(128, 3, 1, border_mode='same')(conv3)
    conv4 = keras.layers.normalization.BatchNormalization()(conv4)
    conv4 = keras.layers.Activation('relu')(conv4)
    
    
    full = keras.layers.pooling.GlobalAveragePooling2D()(conv3)    
    out = keras.layers.Dense(nb_classes, activation='softmax')(full)
    
    
    model = Model(input=x, output=out)
     
    optimizer = keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
     
    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                      patience=50, min_lr=0.0001) 
    hist = model.fit(x_train, Y_train, batch_size=batch_size, nb_epoch=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
    #Print the testing results which has the lowest training loss.
    log = pd.DataFrame(hist.history)
    print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

Using TensorFlow backend.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (10, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (8, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:82: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:92: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 390 samples, validate on 391 samples
Epoch 1/2000
390/390 [==============================] - 24s - loss: 3.6395 - acc: 0.0462 - val_loss: 3.6175 - val_acc: 0.0512
Epoch 2/2000
390/390 [==============================] - 14s - loss: 3.3329 - acc: 0.0897 - val_loss: 4.6517 - val_acc: 0.0281
Epoch 3/2000
390/390 [==============================] - 14s - loss: 3.1507 - acc: 0.1538 - val_loss: 10.8889 - val_acc: 0.0281
Epoch 4/2000
390/390 [==============================] - 13s - loss: 3.0073 - acc: 0.1769 - val_loss: 14.7735 - val_acc: 0.0281
Epoch 5/2000
390/390 [==============================] - 13s - loss: 2.8781 - acc: 0.2051 - val_loss: 15.5421 - val_acc: 0.0281
Epoch 6/2000
390/390 [==============================] - 14s - loss: 2.7307 - acc: 0.2846 - val_loss: 15.6216 - val_acc: 0.0281
Epoch 7/2000
390/390 [==============================] - 14s - loss: 2.6987 - acc: 0.3128 - val_loss: 15.6646 - val_acc: 0.0281
Epoch 8/2000
390/390 [==============================] - 14s - loss:

390/390 [==============================] - 14s - loss: 1.0577 - acc: 0.6897 - val_loss: 1.6331 - val_acc: 0.4476
Epoch 66/2000
390/390 [==============================] - 14s - loss: 1.0596 - acc: 0.6974 - val_loss: 1.4503 - val_acc: 0.4834
Epoch 67/2000
390/390 [==============================] - 14s - loss: 1.0398 - acc: 0.6923 - val_loss: 1.9407 - val_acc: 0.3990
Epoch 68/2000
390/390 [==============================] - 13s - loss: 1.0122 - acc: 0.7051 - val_loss: 1.5333 - val_acc: 0.4731
Epoch 69/2000
390/390 [==============================] - 14s - loss: 0.9763 - acc: 0.7231 - val_loss: 1.0984 - val_acc: 0.6522
Epoch 70/2000
390/390 [==============================] - 14s - loss: 0.9736 - acc: 0.7128 - val_loss: 1.7797 - val_acc: 0.4246
Epoch 71/2000
390/390 [==============================] - 13s - loss: 1.0168 - acc: 0.6821 - val_loss: 1.5848 - val_acc: 0.4680
Epoch 72/2000
390/390 [==============================] - 15s - loss: 0.9720 - acc: 0.7103 - val_loss: 1.7388 - val_acc: 0.424

390/390 [==============================] - 13s - loss: 0.6437 - acc: 0.8154 - val_loss: 1.0186 - val_acc: 0.6343
Epoch 130/2000
390/390 [==============================] - 14s - loss: 0.5966 - acc: 0.8205 - val_loss: 1.5431 - val_acc: 0.5448
Epoch 131/2000
390/390 [==============================] - 13s - loss: 0.7164 - acc: 0.7692 - val_loss: 1.2431 - val_acc: 0.6010
Epoch 132/2000
390/390 [==============================] - 13s - loss: 0.6469 - acc: 0.8051 - val_loss: 0.9782 - val_acc: 0.6419
Epoch 133/2000
390/390 [==============================] - 14s - loss: 0.6604 - acc: 0.7846 - val_loss: 0.8439 - val_acc: 0.7289
Epoch 134/2000
390/390 [==============================] - 14s - loss: 0.6399 - acc: 0.8103 - val_loss: 0.9938 - val_acc: 0.6905
Epoch 135/2000
390/390 [==============================] - 14s - loss: 0.6212 - acc: 0.8026 - val_loss: 1.5249 - val_acc: 0.5320
Epoch 136/2000
390/390 [==============================] - 13s - loss: 0.6556 - acc: 0.7872 - val_loss: 2.5089 - val_acc

390/390 [==============================] - 14s - loss: 0.4862 - acc: 0.8564 - val_loss: 0.8593 - val_acc: 0.6982
Epoch 194/2000
390/390 [==============================] - 14s - loss: 0.4793 - acc: 0.8462 - val_loss: 1.2893 - val_acc: 0.5729
Epoch 195/2000
390/390 [==============================] - 14s - loss: 0.5451 - acc: 0.8179 - val_loss: 1.2888 - val_acc: 0.6317
Epoch 196/2000
390/390 [==============================] - 14s - loss: 0.5343 - acc: 0.8410 - val_loss: 1.2085 - val_acc: 0.6113
Epoch 197/2000
390/390 [==============================] - 12s - loss: 0.5491 - acc: 0.8205 - val_loss: 2.1754 - val_acc: 0.4604
Epoch 198/2000
390/390 [==============================] - 10s - loss: 0.4401 - acc: 0.8667 - val_loss: 1.3023 - val_acc: 0.6061
Epoch 199/2000
390/390 [==============================] - 10s - loss: 0.5241 - acc: 0.8385 - val_loss: 1.2894 - val_acc: 0.6010
Epoch 200/2000
390/390 [==============================] - 10s - loss: 0.5475 - acc: 0.8231 - val_loss: 1.3730 - val_acc

...

390/390 [==============================] - 11s - loss: 0.0163 - acc: 0.9974 - val_loss: 0.6022 - val_acc: 0.8465
Epoch 1720/2000
390/390 [==============================] - 11s - loss: 0.0202 - acc: 0.9949 - val_loss: 0.6550 - val_acc: 0.8389
Epoch 1721/2000
390/390 [==============================] - 11s - loss: 0.0242 - acc: 0.9974 - val_loss: 0.6856 - val_acc: 0.8389
Epoch 1722/2000
390/390 [==============================] - 11s - loss: 0.0205 - acc: 0.9974 - val_loss: 0.6658 - val_acc: 0.8389
Epoch 1723/2000
390/390 [==============================] - 11s - loss: 0.0290 - acc: 0.9923 - val_loss: 0.7121 - val_acc: 0.8338
Epoch 1724/2000
390/390 [==============================] - 11s - loss: 0.0141 - acc: 1.0000 - val_loss: 0.6813 - val_acc: 0.8465
Epoch 1725/2000
390/390 [==============================] - 11s - loss: 0.0142 - acc: 0.9949 - val_loss: 0.6560 - val_acc: 0.8491
Epoch 1726/2000
390/390 [==============================] - 11s - loss: 0.0133 - acc: 1.0000 - val_loss: 0.6208 - 

390/390 [==============================] - 11s - loss: 0.0120 - acc: 1.0000 - val_loss: 0.6140 - val_acc: 0.8465
Epoch 1783/2000
390/390 [==============================] - 11s - loss: 0.0128 - acc: 1.0000 - val_loss: 0.5614 - val_acc: 0.8568
Epoch 1784/2000
390/390 [==============================] - 11s - loss: 0.0114 - acc: 1.0000 - val_loss: 0.5785 - val_acc: 0.8619
Epoch 1785/2000
390/390 [==============================] - 11s - loss: 0.0153 - acc: 1.0000 - val_loss: 0.5766 - val_acc: 0.8491
Epoch 1786/2000
390/390 [==============================] - 11s - loss: 0.0212 - acc: 0.9974 - val_loss: 0.5700 - val_acc: 0.8568
Epoch 1787/2000
390/390 [==============================] - 11s - loss: 0.0213 - acc: 0.9949 - val_loss: 0.5760 - val_acc: 0.8465
Epoch 1788/2000
390/390 [==============================] - 11s - loss: 0.0180 - acc: 1.0000 - val_loss: 0.5865 - val_acc: 0.8619
Epoch 1789/2000
390/390 [==============================] - 11s - loss: 0.0173 - acc: 1.0000 - val_loss: 0.5729 - 

390/390 [==============================] - 11s - loss: 0.0159 - acc: 0.9974 - val_loss: 0.5986 - val_acc: 0.8465
Epoch 1846/2000
390/390 [==============================] - 11s - loss: 0.0207 - acc: 0.9949 - val_loss: 0.7039 - val_acc: 0.8389
Epoch 1847/2000
390/390 [==============================] - 11s - loss: 0.0114 - acc: 1.0000 - val_loss: 0.7180 - val_acc: 0.8440
Epoch 1848/2000
390/390 [==============================] - 11s - loss: 0.0123 - acc: 0.9974 - val_loss: 0.6162 - val_acc: 0.8517
Epoch 1849/2000
390/390 [==============================] - 11s - loss: 0.0108 - acc: 1.0000 - val_loss: 0.7025 - val_acc: 0.8440
Epoch 1850/2000
390/390 [==============================] - 10s - loss: 0.0113 - acc: 1.0000 - val_loss: 0.6322 - val_acc: 0.8542
Epoch 1851/2000
390/390 [==============================] - 11s - loss: 0.0127 - acc: 1.0000 - val_loss: 0.6890 - val_acc: 0.8491
Epoch 1852/2000
390/390 [==============================] - 11s - loss: 0.0092 - acc: 1.0000 - val_loss: 0.6150 - 

390/390 [==============================] - 11s - loss: 0.0087 - acc: 1.0000 - val_loss: 0.5678 - val_acc: 0.8440
Epoch 1909/2000
390/390 [==============================] - 11s - loss: 0.0134 - acc: 1.0000 - val_loss: 0.6038 - val_acc: 0.8414
Epoch 1910/2000
390/390 [==============================] - 11s - loss: 0.0223 - acc: 0.9949 - val_loss: 0.6414 - val_acc: 0.8389
Epoch 1911/2000
390/390 [==============================] - 11s - loss: 0.0123 - acc: 1.0000 - val_loss: 0.5885 - val_acc: 0.8542
Epoch 1912/2000
390/390 [==============================] - 11s - loss: 0.0116 - acc: 1.0000 - val_loss: 0.5973 - val_acc: 0.8465
Epoch 1913/2000
390/390 [==============================] - 11s - loss: 0.0141 - acc: 0.9974 - val_loss: 0.6345 - val_acc: 0.8465
Epoch 1914/2000
390/390 [==============================] - 11s - loss: 0.0141 - acc: 0.9949 - val_loss: 0.5767 - val_acc: 0.8491
Epoch 1915/2000
390/390 [==============================] - 11s - loss: 0.0393 - acc: 0.9821 - val_loss: 0.6287 - 

390/390 [==============================] - 5s - loss: 0.0139 - acc: 1.0000 - val_loss: 0.5786 - val_acc: 0.8517
Epoch 1972/2000
390/390 [==============================] - 5s - loss: 0.0192 - acc: 0.9949 - val_loss: 0.7021 - val_acc: 0.8338
Epoch 1973/2000
390/390 [==============================] - 5s - loss: 0.0181 - acc: 0.9974 - val_loss: 0.6363 - val_acc: 0.8465
Epoch 1974/2000
390/390 [==============================] - 5s - loss: 0.0244 - acc: 0.9923 - val_loss: 0.7626 - val_acc: 0.8286
Epoch 1975/2000
390/390 [==============================] - 5s - loss: 0.0199 - acc: 0.9974 - val_loss: 0.6557 - val_acc: 0.8440
Epoch 1976/2000
390/390 [==============================] - 5s - loss: 0.0196 - acc: 0.9974 - val_loss: 0.6876 - val_acc: 0.8517
Epoch 1977/2000
390/390 [==============================] - 5s - loss: 0.0176 - acc: 0.9974 - val_loss: 0.6656 - val_acc: 0.8593
Epoch 1978/2000
390/390 [==============================] - 6s - loss: 0.0157 - acc: 0.9974 - val_loss: 0.6531 - val_acc: